<a href="https://colab.research.google.com/github/caiocesarcosta/dector_face_yolo_project5/blob/main/dector_face_yolo_project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import json
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import shutil
import yaml
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.preprocessing import image
from google.colab import drive
import matplotlib.pyplot as plt

# Configurações
img_width, img_height = 224, 224 # Redimensiona as imagens para este tamanho
face_detection_threshold = 0.5 # Limiar de confiança para detecção facial
drive.mount('/content/drive', force_remount=True) # Conecta ao google drive para acessar os arquivos

class FaceDetector:
    def __init__(self, img_width):
        self.repoYoloGitHub = 'https://github.com/ultralytics/yolov5'
        self.pretrained_weights = 'yolov5s.pt'  # Usa o modelo pequeno (mais rápido)
        self.img_width = img_width
        self.cloneRepoistoryYoloGitHub()
        self.installDependency()

    def cloneRepoistoryYoloGitHub(self):
        """Clone the yolo repository, from gitHub."""
        print ("Clonando yolo do github")
        if not os.path.exists('/content/yolov5'):
           clone_command = f"git clone {self.repoYoloGitHub}"
           os.system(clone_command)  # Use os.system para executar o comando

    def installDependency(self):
      """Install the dependecies."""
      print ("Instalando dependências")
      if os.path.exists('/content/yolov5'):
        os.chdir('/content/yolov5')
        !pip install -r requirements.txt

    def detectFaces(self, source, conf_thres=0.5):
        """Detecta faces em uma imagem e retorna os bounding boxes."""
        print ("Detectando face da imagem")
        detect_command = f"python detect.py --weights {self.pretrained_weights} --img {self.img_width} --conf {conf_thres} --source \"{source}\" --save-txt"
        os.system(detect_command)  # Use os.system

        return self.parseYOLOResults(source) # carrega para a função as informações da imagem

    def parseYOLOResults(self, source):
        """Analisa os resultados do YOLO e retorna os bounding boxes."""
        results_dir = '/content/yolov5/runs/detect/exp/labels' # Garante que estamos procurando no local correto
        base_name = os.path.basename(source).split('.')[0]
        results_file = os.path.join(results_dir, base_name + '.txt')
        if not os.path.exists(results_file):
            print (f"Aviso: Nenhum objeto detectado {results_dir}!")
            return [] # Retorna uma lista vazia se nenhum arquivo for encontrado

        bounding_boxes = []
        try:
            with open(results_file, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    if len(data) == 5:
                        class_id, x_center, y_center, width, height = map(float, data)
                        bounding_boxes.append((class_id, x_center, y_center, width, height))
        except FileNotFoundError:
            print(f"Arquivo de resultados não encontrado: {results_file}")
        return bounding_boxes

class FaceClassifier:
    def __init__(self, train_dir, validation_dir, img_width, img_height, num_classes, epochs=10, batch_size=32):
      self.train_dir = train_dir
      self.validation_dir = validation_dir
      self.img_width = img_width
      self.img_height = img_height
      self.num_classes = num_classes
      self.epochs = epochs
      self.batch_size = batch_size
      self.model = self.createModel()
      self.compileModel()

    def createModel(self):
        """Cria o modelo de classificação (CNN)."""
        input_shape = (self.img_width, self.img_height, 3)
        model = Sequential([
            Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(self.num_classes, activation='softmax') # Camada de saída
        ])
        return model

    def compileModel(self):
        """Compila o modelo para treinamento."""
        self.model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    def trainModel(self):
        """Treina o modelo usando os dados do Google Drive."""
        train_datagen = ImageDataGenerator(
            rescale=1. / 255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            rotation_range=20)  # Adiciona rotação para aumentar a variabilidade

        test_datagen = ImageDataGenerator(rescale=1. / 255)

        train_generator = train_datagen.flow_from_directory(
            self.train_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=self.batch_size,
            class_mode='categorical')  # Usar categorical para múltiplas classes

        validation_generator = test_datagen.flow_from_directory(
            self.validation_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=self.batch_size,
            class_mode='categorical')  # Usar categorical para múltiplas classes

        self.history = self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // self.batch_size,
            epochs=self.epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // self.batch_size)

        return train_generator.class_indices # retorne o resultado das classes para auxílio
    def evaluateModel(self, validation_generator):
      """Função para avaliar o modelo treinado, e gerar um report."""
      # Depois do treinamento, avaliar o modelo:
      loss, accuracy = self.model.evaluate(validation_generator)
      print(f"Perda (Loss): {loss:.4f}")
      print(f"Precisão (Accuracy): {accuracy:.4f}")

      # Fazer previsões para construir a matriz de confusão e exibir as metricas para a análise do resultado
      from sklearn.metrics import classification_report, confusion_matrix

      val_steps_per_epoch = np.math.ceil(float(validation_generator.samples) / batch_size)
      Y_pred = self.model.predict(validation_generator, steps=val_steps_per_epoch)
      y_pred = np.argmax(Y_pred, axis=1)
      print('Relatório de Classificação')
      print(classification_report(validation_generator.classes, y_pred, target_names=validation_generator.class_indices.keys()))

def crop_face(image, bbox):
    """Corta uma face da imagem usando as coordenadas do bounding box."""
    class_id, x_center, y_center, width_norm, height_norm = bbox
    left = int((x_center - width_norm / 2) * 150)
    top = int((y_center - height_norm / 2) * 150)
    right = int((x_center + width_norm / 2) * 150)
    bottom = int((y_center + height_norm / 2) * 150)
    return image.crop((left, top, right, bottom))

def reconhecerImagem(): # juntando as funções para gerar o treinamento
  """Implementa o reconhecimento facil """
  # Configurações
  train_dir = '/content/drive/MyDrive/the_big_bang_theory/data/train/' # Pasta com as fotos para o treino
  validation_dir = '/content/drive/MyDrive/the_big_bang_theory/data/train/'  # Pasta com as fotos de validação
  img_width, img_height = 224, 224 # Redimensiona as imagens para este tamanho
  num_classes = len(os.listdir(train_dir)) # Ajuste conforme o número de pessoas
  batch_size = 32
  epochs = 5

  # Treinamento do modelo
  # Verifique se há imagens no diretório de treinamento
  if len(os.listdir(train_dir)) == 0:
    print(f"Erro: O diretório de treinamento '{train_dir}' está vazio. Verifique o caminho.")
    return  # Sai da função se o diretório estiver vazio



  # Inicializa o Detector de Faces
  detector = FaceDetector(img_width)
  print("Classe 1 iniciada.")

  # Cria as pastas para o modelo para auxiliar no treinamento
  faces_dir = '/content/coco_yolo/faces'

  #Verifica se a pasta existe, e cria as pastas
  if not os.path.exists(faces_dir):
      os.makedirs(faces_dir, exist_ok = True)

  # Inicializa o Classificador de Faces
  classificacao = FaceClassifier(train_dir, validation_dir, img_width, img_height, num_classes, epochs, batch_size)
  print("Classe 2 iniciada.")

  # Carrega a imagem de exemplo
  #test_image = '/content/yolov5/data/images/bus.jpg'  # imagem de exemplo do yolo
  test_image = '/content/drive/MyDrive/the_big_bang_theory/data/train/penny/penny1.jpg' # imagem que está no google drive

  # Detecta faces na imagem de teste
  bounding_boxes = detector.detectFaces(test_image)

  # Imprimi Bounding boxes encontrados
  print (f"Bounding boxes: {bounding_boxes}")
  # Imagens que as funções não encontrassem o que precisavam, não seriam mais exibidas, já que a detecção seria feita por um novo teste
  if bounding_boxes:
      print("Apresentando Bounding Boxes")
       # Apresenta os dados, caso positivo
      # Carrega a imagem novamente
      img_open = Image.open(test_image)
      print (f"Formato da imagem {img_open.format} e tamanho {img_open.size}")
  else:
      print("Nenhuma anotação foi encontrada nessa imagem, tente outras imagens. O programa se encerrará.")
      return

  # Treinamento do modelo
  class_name = classificacao.trainModel()
  print("Modelo treinado")

  #Apresenta o resultado
  classificacao.evaluateModel(classificacao.validation_generator)
  print("Apresentado Resultado do modelo")

  # Corta as faces e as classifica
  for i, bbox in enumerate(bounding_boxes):
      #Corta a imagem com as dimensões
      face_crop = crop_face(img_open, bbox)
       # Converte para RGB, para evitar erros
      if face_crop.mode != "RGB":
        face_crop = face_crop.convert("RGB")
      #Salva as imagens na pasta
      face_crop.save(os.path.join(faces_dir, f"face_{i}.jpg"))
      print (f"Imagem {i} salva")

#Função para cortar as imagens.
def crop_face(image, bbox):
    """Corta uma face da imagem usando as coordenadas do bounding box."""
    class_id, x_center, y_center, width_norm, height_norm = bbox
    left = int((x_center - width_norm / 2) * 150)
    top = int((y_center - height_norm / 2) * 150)
    right = int((x_center + width_norm / 2) * 150)
    bottom = int((y_center + height_norm / 2) * 150)
    return image.crop((left, top, right, bottom))

reconhecerImagem()


Mounted at /content/drive
Clonando yolo do github
Instalando dependências
Classe 1 iniciada.
Classe 2 iniciada.
Detectando face da imagem
Bounding boxes: [(0.0, 0.495098, 0.517157, 0.980392, 0.955882)]
Apresentando Bounding Boxes
Formato da imagem JPEG e tamanho (204, 204)
Found 56 images belonging to 1 classes.
Found 56 images belonging to 1 classes.
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Modelo treinado


AttributeError: 'FaceClassifier' object has no attribute 'validation_generator'